In [ ]:
import sqlalchemy as sq
import pymysql

import pandas as pd
import numpy as np

import os
import csv

sq.__version__

In [ ]:
import datetime

def convert_unixtime(stamp):
    return datetime.datetime.fromtimestamp(
        int(stamp)
    ).strftime('%Y-%m-%d')

In [ ]:
import getpass

password = getpass.getpass()

In [ ]:
db_connection = "mysql+pymysql://root:{}@192.168.2.117:3306/elgg".format(password)

In [ ]:
engine = sq.create_engine(db_connection,encoding='latin1', echo=True)

In [ ]:
conn = engine.connect()

In [ ]:
engine.connect()

In [ ]:
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.automap import automap_base
Session = sessionmaker(bind=engine)

In [ ]:
Session.configure(bind=engine)
session = Session()

In [ ]:
Base = automap_base()

Base.prepare(engine, reflect=True)

In [ ]:
# Set up mappings

Users = Base.classes.elggusers_entity
Groups = Base.classes.elgggroups_entity
Relationships = Base.classes.elggentity_relationships
Entities = Base.classes.elggentities
MetaData = Base.classes.elggmetadata
MetaStrings = Base.classes.elggmetastrings


### Guide to Elgg Entities

Blogs = Entities(subtype=5)

Group_Members = Users(relationship=member)

Discussions = Entities(subtype=7)

Pages = Entities(subtype=10)

Wire = Entities(subtype=17)

Content = Entities(subtype) -> entity_guid

    Elggmetadata(entity_guid) -> name_id, value_id
    
    Elggmetastrings(name_id OR value_id)
    
#Comments

Blog is container entity - GUID = blog guid

Blog guid = 10

search container for blog guid, return container guid

elggmetadata(container_guid)

Elggmetastrings(name_id OR value_id)

#Skills

user_GUID -> elggmetadata(container_guid) - name_id = 60

In [ ]:
users = session.query(Users).all()
user_data = session.query(Entities).filter(Entities.type == 'user')

In [ ]:
# Set up dictionaries and lists

gcc_users = {}
dept_dict = {}

In [ ]:
data_path = r'/Users/toferc/Documents/Data/'

In [ ]:
with open(os.path.join(data_path, 'csv_keys.csv'), "r") as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)
    
    for row in reader:
        email, acronym = row
        dept_dict[email] = acronym

dept_dict['cadets.gc.ca'] = 'CADETS'
dept_dict['canada.gc.ca'] = 'CANADA'
dept_dict['canada.ca'] = 'CANADA'
dept_dict['tribunal.gc.ca'] = 'TRIBUNAL'
dept_dict['cannor.gc.ca'] = 'CED/DEC'
dept_dict['ci-oic.gc.ca'] = 'CI/OIC'
dept_dict['ccgs-ngcc.gc.ca'] = 'CCGS/NGCC'
dept_dict['god.ccgs-ngcc.gc.ca'] = 'CCGS/NGCC'
dept_dict['clo-ocol.gc.ca'] = 'OCOL/CLO'
dept_dict['csps.gc.ca'] = 'CSPS/EFPC'
dept_dict['interenational.gc.ca'] = 'DFAITD/MAECD'
dept_dict['cnb-ncw.gc.ca'] = 'CNB/NCW'
dept_dict['ncw-cnb.gc.ca'] = 'CNB/NCW'
dept_dict['nfb.gc.ca'] = 'NFB/ONF'
dept_dict['nrccan-rncan.gc.ca'] = 'NRCAN/RNCAN'
dept_dict['nserc-crsng.gc.ca'] = 'NSERC/CRSNG'
dept_dict['pbc-clcc.gc.ca'] = 'PBC/CLCC'
dept_dict['pco.bcp.gc.ca'] = 'PCO/BCP'
dept_dict['pipsc.ca'] = 'PIPSC/IPFPC'
dept_dict['ps.sp.gc.ca'] = 'PS/SP'
dept_dict['servicecanada.gc.ca.gc.ca'] = 'HRSDC/RHDSC'
dept_dict['fintrac-canafe.gc.ca'] = 'FINTRAC'
dept_dict['gmail.com'] = 'GMAIL'
dept_dict['tribunbal.gc.ca'] = 'TRIBUNAL'

In [ ]:
dept_dict

In [ ]:
for user, user_d in zip(users, user_data):
    gcc_users[user.guid] = {}
    gcc_users[user.guid]['name'] = user.name
    gcc_users[user.guid]['id'] = user.guid
    gcc_users[user.guid]['joined'] = convert_unixtime(user_d.time_created)
    email = user.email
    ampersand = email.find('@')
    tail = email[ampersand + 1:]
    try:
        gcc_users[user.guid]['department'] = dept_dict[tail.lower()]
    except KeyError:
        gcc_users[user.guid]['department'] = 'ERROR'

In [ ]:
gcc_users

In [ ]:
friends = session.query(Relationships).filter(Relationships.relationship == 'friend')

In [ ]:
edges = []

for f in friends:
    if f.time_created == 0:
        created = '2008-12-15'
    else:
        created = convert_unixtime(f.time_created)
        
    edges.append([f.guid_one, f.guid_two, created])

In [ ]:
edges[100000:100010]

In [ ]:
import networkx as nx

In [ ]:
G = nx.DiGraph()

In [ ]:
for user in gcc_users:
    G.add_node(gcc_users[user]['id'], 
               name=gcc_users[user]['name'], 
               department=gcc_users[user]['department'],
               joined=gcc_users[user]['joined']
              )

In [ ]:
for edge in edges:
    source, target, created = edge
    G.add_edge(source, target, attr_dict={'created':created})

In [ ]:
nx.write_gexf(G, "gcconnex_test_2016_03_02.gexf")